In [1]:
#a simple script to test loading the bias model into pytorch
#we have some scripts stored in bpnet lite
import sys
sys.path.append('/data/leslie/sarthak/chrombpnet')
from bpnetlite.bpnet import BPNet


In [2]:
#now let's try loading the model using the model method
model = BPNet.from_chrombpnet('/data/leslie/sarthak/data/chrombpnet_test/bias_model/models/k562_bias.h5')

In [3]:
model

TypeError: object of type 'int' has no len()

In [4]:
print(model.__class__) #this does work
print(vars(model))  # Print the __dict__ attribute of the model


<class 'bpnetlite.bpnet.BPNet'>


TypeError: object of type 'int' has no len()

In [5]:
def inspect_attributes(obj):
    for attr, value in obj.__dict__.items():
        try:
            print(f"{attr}: {value}")
        except TypeError as e:
            print(f"Error printing {attr}: {e}")

inspect_attributes(model)

training: True
_parameters: OrderedDict()
_buffers: OrderedDict()
_non_persistent_buffers_set: set()
_backward_pre_hooks: OrderedDict()
_backward_hooks: OrderedDict()
_is_full_backward_hook: None
_forward_hooks: OrderedDict()
_forward_hooks_with_kwargs: OrderedDict()
_forward_hooks_always_called: OrderedDict()
_forward_pre_hooks: OrderedDict()
_forward_pre_hooks_with_kwargs: OrderedDict()
_state_dict_hooks: OrderedDict()
_state_dict_pre_hooks: OrderedDict()
_load_state_dict_pre_hooks: OrderedDict()
_load_state_dict_post_hooks: OrderedDict()
Error printing _modules: object of type 'int' has no len()
n_filters: 128
n_layers: 4
n_outputs: 1
n_control_tracks: 0
alpha: 1
name: bpnet.128.4
trimming: 557
logger: <bpnetlite.logging.Logger object at 0x7f1396422570>


In [7]:
#let's first load the model without any weights
model = BPNet()
model #ok this seems fine!

BPNet(
  (iconv): Conv1d(4, 64, kernel_size=(21,), stride=(1,), padding=(10,))
  (irelu): ReLU()
  (rconvs): ModuleList(
    (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
    (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
    (5): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
    (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
    (7): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(256,), dilation=(256,))
  )
  (rrelus): ModuleList(
    (0-7): 8 x ReLU()
  )
  (fconv): Conv1d(66, 2, kernel_size=(75,), stride=(1,), padding=(37,))
  (linear): Linear(in_features=65, out_features=1, bias=True)
)

In [8]:
model._modules

OrderedDict([('iconv',
              Conv1d(4, 64, kernel_size=(21,), stride=(1,), padding=(10,))),
             ('irelu', ReLU()),
             ('rconvs',
              ModuleList(
                (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
                (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
                (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
                (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
                (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
                (5): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
                (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
                (7): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(256,), dilation=(256,))
              )),
             ('rre

In [9]:
#let's go through this step by step
import torch
import h5py
filename = '/data/leslie/sarthak/data/chrombpnet_test/bias_model/models/k562_bias.h5'

h5 = h5py.File(filename, "r")
w = h5['model_weights']

print(w.keys())

<KeysViewHDF5 ['add', 'add_1', 'add_2', 'add_3', 'bpnet_1conv', 'bpnet_1crop', 'bpnet_1st_conv', 'bpnet_2conv', 'bpnet_2crop', 'bpnet_3conv', 'bpnet_3crop', 'bpnet_4conv', 'bpnet_4crop', 'gap', 'logcount_predictions', 'logits_profile_predictions', 'logits_profile_predictions_preflatten', 'prof_out_precrop', 'sequence', 'top_level_model_weights']>


In [11]:
print(len(w.keys()))

20


In [15]:
if 'bpnet_1conv' in w.keys():
    prefix = ""
else:
    prefix = "wo_bias_"

namer = lambda prefix, suffix: '{0}{1}/{0}{1}'.format(prefix, suffix)
k, b = 'kernel:0', 'bias:0'

n_layers = 0
for layer_name in w.keys():
    try:
        idx = int(layer_name.split("_")[-1].replace("conv", ""))
        print(layer_name, idx)
        n_layers = max(n_layers, idx)
    except:
        print('layer_name:', layer_name) #ahh, several of them come here and there's some issue here...
        pass
print(n_layers) #does correctly find that there's 4 layers so this is not the issue

layer_name: add
add_1 1
add_2 2
add_3 3
bpnet_1conv 1
layer_name: bpnet_1crop
layer_name: bpnet_1st_conv
bpnet_2conv 2
layer_name: bpnet_2crop
bpnet_3conv 3
layer_name: bpnet_3crop
bpnet_4conv 4
layer_name: bpnet_4crop
layer_name: gap
layer_name: logcount_predictions
layer_name: logits_profile_predictions
layer_name: logits_profile_predictions_preflatten
layer_name: prof_out_precrop
layer_name: sequence
layer_name: top_level_model_weights
4


In [16]:
name = namer(prefix, "bpnet_1conv")
n_filters = w[name][k].shape[2]
print(name)
print(n_filters)

model = BPNet(n_layers=n_layers, n_filters=n_filters, n_outputs=1,
    n_control_tracks=0, trimming=(2114-1000)//2)
print(model) #this loads just fine!

bpnet_1conv/bpnet_1conv
128
BPNet(
  (iconv): Conv1d(4, 128, kernel_size=(21,), stride=(1,), padding=(10,))
  (irelu): ReLU()
  (rconvs): ModuleList(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
  )
  (rrelus): ModuleList(
    (0-3): 4 x ReLU()
  )
  (fconv): Conv1d(128, 1, kernel_size=(75,), stride=(1,), padding=(37,))
  (linear): Linear(in_features=128, out_features=1, bias=True)
)


In [17]:
convert_w = lambda x: torch.nn.Parameter(torch.tensor(
    x[:]).permute(2, 1, 0))
convert_b = lambda x: torch.nn.Parameter(torch.tensor(x[:]))

iname = namer(prefix, 'bpnet_1st_conv')

model.iconv.weight = convert_w(w[iname][k])
model.iconv.bias = convert_b(w[iname][b])
model.iconv.padding = (21 - 1) // 2

In [18]:
model #ahh here it is we found the issue, let's investigate

TypeError: object of type 'int' has no len()

In [21]:
model = BPNet(n_layers=n_layers, n_filters=n_filters, n_outputs=1,
    n_control_tracks=0, trimming=(2114-1000)//2)
print(model.iconv)

Conv1d(4, 128, kernel_size=(21,), stride=(1,), padding=(10,))


In [20]:
convert_w(w[iname][k]).shape

torch.Size([128, 4, 21])

In [23]:
model.iconv.weight = convert_w(w[iname][k])
print(model.iconv)
print(model)

Conv1d(4, 128, kernel_size=(21,), stride=(1,), padding=(10,))
BPNet(
  (iconv): Conv1d(4, 128, kernel_size=(21,), stride=(1,), padding=(10,))
  (irelu): ReLU()
  (rconvs): ModuleList(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
  )
  (rrelus): ModuleList(
    (0-3): 4 x ReLU()
  )
  (fconv): Conv1d(128, 1, kernel_size=(75,), stride=(1,), padding=(37,))
  (linear): Linear(in_features=128, out_features=1, bias=True)
)


In [27]:
#that is also fine, is it th enext one?
print(convert_b(w[iname][b]).shape)
print(model.iconv.bias.shape)

torch.Size([128])
torch.Size([128])


In [28]:
model.iconv.bias = convert_b(w[iname][b])
print(model)

BPNet(
  (iconv): Conv1d(4, 128, kernel_size=(21,), stride=(1,), padding=(10,))
  (irelu): ReLU()
  (rconvs): ModuleList(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
  )
  (rrelus): ModuleList(
    (0-3): 4 x ReLU()
  )
  (fconv): Conv1d(128, 1, kernel_size=(75,), stride=(1,), padding=(37,))
  (linear): Linear(in_features=128, out_features=1, bias=True)
)


In [30]:
#so it's this last thing?
print(model.iconv.padding)
print((21 - 1) // 2)

#it's because it's a fuckign int instead of a tuple... wtf??
#


(10,)
10


In [31]:




for i in range(1, n_layers+1):
    lname = namer(prefix, 'bpnet_{}conv'.format(i))

    model.rconvs[i-1].weight = convert_w(w[lname][k])
    model.rconvs[i-1].bias = convert_b(w[lname][b])

prefix = prefix + "bpnet_" if prefix != "" else ""

fname = namer(prefix, 'prof_out_precrop')
model.fconv.weight = convert_w(w[fname][k])
model.fconv.bias = convert_b(w[fname][b])
model.fconv.padding = (75 - 1) // 2

name = namer(prefix, "logcount_predictions")
model.linear.weight = torch.nn.Parameter(torch.tensor(w[name][k][:].T))
model.linear.bias = convert_b(w[name][b])

In [34]:
model

TypeError: object of type 'int' has no len()

In [35]:
#there's probalby another similar error...
model = BPNet(n_layers=n_layers, n_filters=n_filters, n_outputs=1,
    n_control_tracks=0, trimming=(2114-1000)//2)

for i in range(1, n_layers+1):
    lname = namer(prefix, 'bpnet_{}conv'.format(i))

    model.rconvs[i-1].weight = convert_w(w[lname][k])
    model.rconvs[i-1].bias = convert_b(w[lname][b])
    
model

BPNet(
  (iconv): Conv1d(4, 128, kernel_size=(21,), stride=(1,), padding=(10,))
  (irelu): ReLU()
  (rconvs): ModuleList(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
  )
  (rrelus): ModuleList(
    (0-3): 4 x ReLU()
  )
  (fconv): Conv1d(128, 1, kernel_size=(75,), stride=(1,), padding=(37,))
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [36]:
fname = namer(prefix, 'prof_out_precrop')
model.fconv.weight = convert_w(w[fname][k])
model.fconv.bias = convert_b(w[fname][b])
# model.fconv.padding = (75 - 1) // 2 #bro it's this fucking int again
model

BPNet(
  (iconv): Conv1d(4, 128, kernel_size=(21,), stride=(1,), padding=(10,))
  (irelu): ReLU()
  (rconvs): ModuleList(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
  )
  (rrelus): ModuleList(
    (0-3): 4 x ReLU()
  )
  (fconv): Conv1d(128, 1, kernel_size=(75,), stride=(1,), padding=(37,))
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [37]:
name = namer(prefix, "logcount_predictions")
model.linear.weight = torch.nn.Parameter(torch.tensor(w[name][k][:].T))
model.linear.bias = convert_b(w[name][b])

In [38]:
model

BPNet(
  (iconv): Conv1d(4, 128, kernel_size=(21,), stride=(1,), padding=(10,))
  (irelu): ReLU()
  (rconvs): ModuleList(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
  )
  (rrelus): ModuleList(
    (0-3): 4 x ReLU()
  )
  (fconv): Conv1d(128, 1, kernel_size=(75,), stride=(1,), padding=(37,))
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [40]:
print(model.fconv.padding)
print(74//2)

(37,)
37


# let's fix it and run it with hyena again and just comment out those 2 line!

In [1]:
#a simple script to test loading the bias model into pytorch
#we have some scripts stored in bpnet lite
import sys
sys.path.append('/data/leslie/sarthak/chrombpnet')
from bpnetlite.bpnet import BPNet

#now let's try loading the model using the model method
model = BPNet.from_chrombpnet('/data/leslie/sarthak/data/chrombpnet_test/bias_model/models/k562_bias.h5')
model

BPNet(
  (iconv): Conv1d(4, 128, kernel_size=(21,), stride=(1,), padding=(10,))
  (irelu): ReLU()
  (rconvs): ModuleList(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
  )
  (rrelus): ModuleList(
    (0-3): 4 x ReLU()
  )
  (fconv): Conv1d(128, 1, kernel_size=(75,), stride=(1,), padding=(37,))
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [6]:
#and this works, just had to comment out those 2 lines lol!
#let's make a random size input
import torch
data = torch.rand(64,4,2114)*10
out = model(data)
print(len(out))
print(out[0].shape) #the profile
print(out[1].shape) #the counts
print(out)

2
torch.Size([64, 1, 1000])
torch.Size([64, 1])
(tensor([[[ 16.5500, -20.1725,  12.6976,  ...,  -2.3728,  11.5527,  -7.2329]],

        [[ 13.6432,  -0.7343,  20.7340,  ...,   3.0503,  18.0686,  18.2985]],

        [[ 10.6004,  -5.5268,  -2.5166,  ...,  26.0202,  10.9781,   1.2610]],

        ...,

        [[ 12.5137,   2.9826,  14.4564,  ...,  -3.8280,   5.0520,   7.3202]],

        [[ -6.4231,  -5.2557,  13.0977,  ...,  22.6658,  18.4078,  -2.7688]],

        [[ 25.8086,  -3.8405,  14.2526,  ...,  -7.3776,  22.0719,   4.5368]]],
       grad_fn=<SliceBackward0>), tensor([[17.0681],
        [16.6567],
        [17.0603],
        [16.4891],
        [16.4478],
        [16.5447],
        [16.7957],
        [16.7515],
        [16.3491],
        [16.7141],
        [16.7493],
        [16.7669],
        [16.9877],
        [16.2577],
        [16.9937],
        [16.1349],
        [16.6623],
        [16.8176],
        [16.5313],
        [16.6301],
        [16.9759],
        [16.4674],
        [16

In [12]:
import torch
data = torch.rand(64,4,4114)*10
out = model(data)
print(len(out))
print(out[0].shape) #the profile
print(out[1].shape) #the counts
print(out) #works as long as it is able to crop out a certian amount! In this case crops out a total of 2114, but we can likely adjust this somewhere in the model to be 800!

2
torch.Size([64, 1, 3000])
torch.Size([64, 1])
(tensor([[[ 14.5144,  10.0297,   4.6547,  ...,  -5.2196,  15.5950,  16.0039]],

        [[  6.9590,   6.0129,   0.6853,  ...,   8.6671,  -5.7772,  24.9934]],

        [[  4.4261,  18.9802, -14.9654,  ...,   1.6526,   9.3276,  -7.7576]],

        ...,

        [[  5.3253,  15.5626,   5.4877,  ...,  21.5688,   5.2422,   9.4834]],

        [[ -9.1263,  17.5073,  22.9212,  ...,  15.4544,   8.4311,   3.6903]],

        [[  9.9269,   5.3502,  -2.9786,  ...,   2.7305,  -2.3351,  17.0411]]],
       grad_fn=<SliceBackward0>), tensor([[16.6574],
        [16.8335],
        [16.6536],
        [16.6040],
        [16.8681],
        [16.6982],
        [16.5354],
        [16.6358],
        [16.7911],
        [16.4574],
        [16.6465],
        [16.6201],
        [16.7730],
        [16.4544],
        [16.6272],
        [16.4799],
        [16.5811],
        [16.8360],
        [16.6037],
        [16.5485],
        [16.7343],
        [16.8720],
        [16

In [17]:
model.eval()
data = torch.tensor(range(2114), dtype=torch.float32).view(1,1,2114)
#now make it 64 x 4 x 2114
data = data.repeat(64,4,1)
print(data.shape)
out = model(data)
print(len(out))
print(out[0].shape) #the profile
print(out[1].shape) #the counts
print(out)
#now it's a big question of if we can see similar results for the same input in the 

torch.Size([64, 4, 2114])
2
torch.Size([64, 1, 1000])
torch.Size([64, 1])
(tensor([[[ -965.9982,  -967.7639,  -969.5287,  ..., -2726.4048,
          -2728.1697, -2729.9338]],

        [[ -965.9982,  -967.7639,  -969.5287,  ..., -2726.4048,
          -2728.1697, -2729.9338]],

        [[ -965.9982,  -967.7639,  -969.5287,  ..., -2726.4048,
          -2728.1697, -2729.9338]],

        ...,

        [[ -965.9982,  -967.7639,  -969.5287,  ..., -2726.4048,
          -2728.1697, -2729.9338]],

        [[ -965.9982,  -967.7639,  -969.5287,  ..., -2726.4048,
          -2728.1697, -2729.9338]],

        [[ -965.9982,  -967.7639,  -969.5287,  ..., -2726.4048,
          -2728.1697, -2729.9338]]], grad_fn=<SliceBackward0>), tensor([[3082.0576],
        [3082.0576],
        [3082.0576],
        [3082.0576],
        [3082.0576],
        [3082.0576],
        [3082.0576],
        [3082.0576],
        [3082.0576],
        [3082.0576],
        [3082.0576],
        [3082.0576],
        [3082.0576],
     

# testing tensorflow model and seeing how similar the input is

In [1]:
#we use the chrombpnet environment and get the model here
import argparse
import sys
sys.path.append('/data/leslie/sarthak/chrombpnet/chrombpnet/')
from test_args import main

sys.argv = [
    'test_args.py', 'pipeline',
    '-ibam', 'merged.bam',
    '-d', 'ATAC',
    '-g', 'hg38.fa',
    '-c', 'hg38.chrom.sizes',
    '-p', 'peaks_no_blacklist.bed',
    '-n', 'output_negatives.bed',
    '-fl', 'splits/fold_0.json',
    '-b', 'bias_model/models/k562_bias.h5',
    '-o', 'chrombpnet_model/'
]
args = main()
#and now we can simulate our args

#now we modify args
import json
import os
if args.file_prefix:
    fpx = args.file_prefix+"_"
else:
    fpx = ""
    
# Shift bam and convert to bigwig
# import chrombpnet.helpers.preprocessing.reads_to_bigwig as reads_to_bigwig	
args.output_prefix = os.path.join(args.output_dir,"auxiliary/{}data".format(fpx))
args.plus_shift = None
args.minus_shift = None
args.bigwig = os.path.join(args.output_dir,"auxiliary/{}data_unstranded.bw".format(fpx))
args.output_prefix = os.path.join(args.output_dir,"evaluation/{}bw_shift_qc".format(fpx))
os.chdir('/data/leslie/sarthak/data/chrombpnet_test/')
folds = json.load(open(args.chr_fold_path))
assert(len(folds["valid"]) > 0) # validation list of chromosomes is empty
args.chr = folds["valid"][0]
args.pwm_width=24
folds
import copy
import chrombpnet.training.models.chrombpnet_with_bias_model as chrombpnet_with_bias_model
import chrombpnet.training.train as train
args_copy = copy.deepcopy(args)
if args_copy.architecture_from_file is None:
    args_copy.architecture_from_file = 	chrombpnet_with_bias_model.__file__
args_copy.peaks = os.path.join(args.output_dir,"auxiliary/{}filtered.peaks.bed".format(fpx))
args_copy.nonpeaks = os.path.join(args.output_dir,"auxiliary/{}filtered.nonpeaks.bed".format(fpx))
args_copy.output_prefix = os.path.join(args.output_dir,"models/{}chrombpnet".format(fpx))
args_copy.params = os.path.join(args.output_dir,"logs/{}chrombpnet_model_params.tsv".format(fpx))
args_copy #this is the official one that has all of the things specified!
args = args_copy
#but instead of directly calling train.main, we'll go through it to get to the source of our things!
import numpy as np
import chrombpnet.training.data_generators.initializers as initializers
parameters = train.get_model_param_dict(args)
print(parameters)
np.random.seed(args.seed)

# get model architecture to load
# model, architecture_module=train.get_model(args, parameters)

# initialize generators to load data
# train_generator = initializers.initialize_generators(args, "train", parameters, return_coords=False) #gets the peaks and takes forever to get the sequences
# valid_generator = initializers.initialize_generators(args, "valid", parameters, return_coords=False)
model, architecture_module=train.get_model(args, parameters) #this is killing it for some reason if we use the lt nodes...

Namespace(cmd='pipeline', genome='hg38.fa', chrom_sizes='hg38.chrom.sizes', input_bam_file='merged.bam', input_fragment_file=None, input_tagalign_file=None, output_dir='chrombpnet_model/', data_type='ATAC', peaks='peaks_no_blacklist.bed', nonpeaks='output_negatives.bed', chr_fold_path='splits/fold_0.json', outlier_threshold=0.9999, ATAC_ref_path=None, DNASE_ref_path=None, num_samples=10000, inputlen=2114, outputlen=1000, seed=1234, epochs=50, early_stop=5, learning_rate=0.001, trackables=['logcount_predictions_loss', 'loss', 'logits_profile_predictions_loss', 'val_logcount_predictions_loss', 'val_loss', 'val_logits_profile_predictions_loss'], architecture_from_file=None, file_prefix=None, html_prefix='./', bsort=False, tmpdir=None, no_st=False, skip_preprocessing=False, save_data=False, bias_model_path='bias_model/models/k562_bias.h5', negative_sampling_ratio=0.1, filters=512, n_dilation_layers=8, max_jitter=500, batch_size=64)
{'counts_loss_weight': '76.2', 'filters': '512', 'n_dil_la

2024-06-11 15:04:51.278776: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 15:04:57.437697: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9459 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3b:00.0, compute capability: 7.5
2024-06-11 15:04:57.447099: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9459 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:5e:00.0, compute capability: 7.5
2024-06-11 15:04:57.449117: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /j

got the model


In [2]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 2114, 4)]    0           []                               
                                                                                                  
 model_wo_bias (Functional)     [(None, 1000),       6377986     ['sequence[0][0]']               
                                 (None, 1)]                                                       
                                                                                                  
 model (Functional)             [(None, 1000),       217730      ['sequence[0][0]']               
                                 (None, 1)]                                                       
                                                                                              

In [11]:
#let's have it be the same input
import numpy as np
data = np.array(range(2114)).reshape(1, 1, 2114)
data = np.repeat(data, 64, axis=0)
data = np.repeat(data, 4, axis=1)
data = data.astype(np.float32)
#and reshape it to be 64 x 2114 x 4
data = np.transpose(data, (0,2,1))
print(data.shape)
#now put it through the model
out = model.predict(data)

(64, 2114, 4)


In [12]:
print(out[0].shape)
print(out[1].shape)
print(out)

(64, 1000)
(64, 1)
[array([[-2236.4678, -2240.0198, -2243.559 , ..., -5759.782 , -5763.3086,
        -5766.836 ],
       [-2236.4678, -2240.0198, -2243.559 , ..., -5759.782 , -5763.3086,
        -5766.836 ],
       [-2236.4678, -2240.0198, -2243.559 , ..., -5759.782 , -5763.3086,
        -5766.836 ],
       ...,
       [-2236.4678, -2240.0198, -2243.559 , ..., -5759.782 , -5763.3086,
        -5766.836 ],
       [-2236.4678, -2240.0198, -2243.559 , ..., -5759.782 , -5763.3086,
        -5766.836 ],
       [-2236.4678, -2240.0198, -2243.559 , ..., -5759.782 , -5763.3086,
        -5766.836 ]], dtype=float32), array([[4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       [4647.6807],
       

In [13]:
out = model(data)
print(out[0].shape)
print(out[1].shape)
print(out)

(64, 1000)
(64, 1)
[<tf.Tensor: shape=(64, 1000), dtype=float32, numpy=
array([[-2236.4683, -2240.02  , -2243.56  , ..., -5759.7803, -5763.312 ,
        -5766.833 ],
       [-2236.4683, -2240.02  , -2243.56  , ..., -5759.7803, -5763.312 ,
        -5766.833 ],
       [-2236.4683, -2240.02  , -2243.56  , ..., -5759.7803, -5763.312 ,
        -5766.833 ],
       ...,
       [-2236.4683, -2240.02  , -2243.56  , ..., -5759.7803, -5763.312 ,
        -5766.833 ],
       [-2236.4683, -2240.02  , -2243.56  , ..., -5759.7803, -5763.312 ,
        -5766.833 ],
       [-2236.4683, -2240.02  , -2243.56  , ..., -5759.7803, -5763.312 ,
        -5766.833 ]], dtype=float32)>, <tf.Tensor: shape=(64, 1), dtype=float32, numpy=
array([[4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68],
       [4647.68

In [14]:
#wait we expect them to give different outputs we're comparing the output of the bias model to the full chrombpnet model!! Let's redo it but this time only load the bias model!!

submodel_layer = model.get_layer('model')
submodel_layer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 2114, 4)]    0           []                               
                                                                                                  
 bpnet_1st_conv (Conv1D)        (None, 2094, 128)    10880       ['sequence[0][0]']               
                                                                                                  
 bpnet_1conv (Conv1D)           (None, 2090, 128)    49280       ['bpnet_1st_conv[0][0]']         
                                                                                                  
 bpnet_1crop (Cropping1D)       (None, 2090, 128)    0           ['bpnet_1st_conv[0][0]']         
                                                                                              

In [16]:
out = submodel_layer.predict(data)
print(out[0].shape)
print(out[1].shape)
print(out)

(64, 1000)
(64, 1)
[array([[ -965.99817,  -967.7634 ,  -969.5292 , ..., -2726.4028 ,
        -2728.168  , -2729.9333 ],
       [ -965.99817,  -967.7634 ,  -969.5292 , ..., -2726.4028 ,
        -2728.168  , -2729.9333 ],
       [ -965.99817,  -967.7634 ,  -969.5292 , ..., -2726.4028 ,
        -2728.168  , -2729.9333 ],
       ...,
       [ -965.99817,  -967.7634 ,  -969.5292 , ..., -2726.4028 ,
        -2728.168  , -2729.9333 ],
       [ -965.99817,  -967.7634 ,  -969.5292 , ..., -2726.4028 ,
        -2728.168  , -2729.9333 ],
       [ -965.99817,  -967.7634 ,  -969.5292 , ..., -2726.4028 ,
        -2728.168  , -2729.9333 ]], dtype=float32), array([[3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [3082.5818],
       [308